# Capítulo 08: Integração Pandas e Polars

## Tópicos Abordados:
- Conversões Pandas ↔ Arrow
- Conversões Polars ↔ Arrow
- Interoperabilidade DuckDB
- Otimizações de tipos
- Pipelines híbridos


In [ ]:
# Setup e Importações
import pyarrow as pa
import duckdb
import pandas as pd
import numpy as np

# Configuração de display do pandas
pd.set_option('display.max_rows', 10)

print('Bibliotecas importadas com sucesso!')

In [ ]:
# Criação de dados de exemplo para o capítulo
data = pa.table({
    'id': range(10000),
    'valor': np.random.randn(10000),
    'categoria': np.random.choice(['A', 'B', 'C'], 10000)
})

print(f'Dataset criado com {len(data)} registros')
print(data.schema)

## Conversões Pandas ↔ Arrow
Exemplos e exercícios sobre conversões pandas ↔ arrow.

In [ ]:
# Prática: Conversões Pandas ↔ Arrow
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()

## Conversões Polars ↔ Arrow
Exemplos e exercícios sobre conversões polars ↔ arrow.

In [ ]:
# Prática: Conversões Polars ↔ Arrow
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()

## Interoperabilidade DuckDB
Exemplos e exercícios sobre interoperabilidade duckdb.

In [ ]:
# Prática: Interoperabilidade DuckDB
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()

## Otimizações de tipos
Exemplos e exercícios sobre otimizações de tipos.

In [ ]:
# Prática: Otimizações de tipos
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()

## Pipelines híbridos
Exemplos e exercícios sobre pipelines híbridos.

In [ ]:
# Prática: Pipelines híbridos
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()